<a href="https://colab.research.google.com/github/123Prostab/Projects/blob/main/pytorch_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn


In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
from tqdm.asyncio import trange
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
batch_size = 64
#create data loaders
train_dataloader = DataLoader(training_data,batch_size = batch_size)
test_dataloader = DataLoader(test_data,batch_size = batch_size)
for X,y in test_dataloader:
  print(f'shape of X: {X.shape}')
  print(f'shape of y:{y.shape}{y.dtype}')
  break

shape of X: torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64


In [18]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
print(f"using {device} device")

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork,self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits


model = NeuralNetwork().to(device)
print(model)







using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-3)

In [20]:
train_dataloader

In [21]:
enumerate(train_dataloader)

In [22]:
for batch , (X,y) in enumerate(train_dataloader):
  print(batch,(X,y))

Streaming output truncated to the last 5000 lines.
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0314, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0157, 0.0000],
          ...,
          [0.0000, 0.0627, 0.3922,  ..., 0.5255, 0.8745, 0.0000],
          [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.2000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        ...,


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0118, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.4314, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.4118, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.1333, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,

In [23]:
def train(dataloader,model,loss_fn,optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch , (X,y) in enumerate(dataloader):
    X,y = X.to(device),y.to(device)

    #compute pred error
    pred = model(X)
    loss = loss_fn(pred,y)
    #back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if batch % 100 == 0:
      loss,current = loss.item(),batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader,model,loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0 , 0
  with torch.no_grad():
    for X,y in dataloader:
      X, y = X.to(device),y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred,y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.298200  [    0/60000]
loss: 2.286051  [ 6400/60000]
loss: 2.267366  [12800/60000]
loss: 2.267066  [19200/60000]
loss: 2.244748  [25600/60000]
loss: 2.214301  [32000/60000]
loss: 2.229787  [38400/60000]
loss: 2.192432  [44800/60000]
loss: 2.197627  [51200/60000]
loss: 2.160098  [57600/60000]
Test Error: 
 Accuracy: 43.2%, Avg loss: 2.155977 

Epoch 2
-------------------------------
loss: 2.169374  [    0/60000]
loss: 2.157834  [ 6400/60000]
loss: 2.102403  [12800/60000]
loss: 2.112889  [19200/60000]
loss: 2.065650  [25600/60000]
loss: 2.006227  [32000/60000]
loss: 2.032113  [38400/60000]
loss: 1.957429  [44800/60000]
loss: 1.966738  [51200/60000]
loss: 1.876519  [57600/60000]
Test Error: 
 Accuracy: 59.9%, Avg loss: 1.885507 

Epoch 3
-------------------------------
loss: 1.923011  [    0/60000]
loss: 1.891757  [ 6400/60000]
loss: 1.777620  [12800/60000]
loss: 1.805363  [19200/60000]
loss: 1.705656  [25600/60000]
loss: 1.654138  [32000/600

In [26]:
torch.save(model.state_dict(),"model.pth")
print("Saved pytorch model to model.pth ")


Saved pytorch model to model.pth 


In [27]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]


In [28]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [29]:
X,y  = test_data[0][0],test_data[0][1]
with torch.no_grad():
  pred = model(X)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Ankle boot", Actual: "Ankle boot"
